# EG 24/25 - Conversão do TPC1 para Lark

## Maria Cunha e Tomás Campinho - Gr10

Link Google Colab: https://colab.research.google.com/drive/1mEA8PuKzugD_vmJPzaZfyOX06TzLaCsB?usp=sharing

## Implementar e adaptar a seguinte gramática em Lark:

Terminal: { '.', ';', '[', ']', num }  
Non-Terminal: { S, Is, RI, I }  
Production Rules (P):  

- p1: Sentence: Signal Intervals '.'  

- p2: Signal: '+'  

- p3: Signal: '-'  

- p4: Intervals: Interval  

- p5: Intervals: Intervals Interval  

- p6: Interval: '[' num ':' num ']'  

  - CC1: p[4] > p[2] &  
  - CC2: p[2] >= parser.anterior  
  - parser.anterior = p[4]  
  - parser.erro = not (CC1) or not (CC2)  


In [2]:
!pip install lark

  Using cached lark-1.2.2-py3-none-any.whl.metadata (1.8 kB)
Using cached lark-1.2.2-py3-none-any.whl (111 kB)


In [ ]:
from lark import Lark, Transformer

grammar = r'''
// Regras Sintáticas
sentence: signal intervals DOT
intervals: interval+
signal: PLUS
      | MINUS
interval: PE num COL num PD

// Regras Lexicográficas
num: /\d+(\.\d+)?/
PLUS: "+"
MINUS: "-"
DOT: "."
PE:"["
PD:"]"
COL:":"

// Ignorar espaços em branco
%import common.WS
%ignore WS
'''

parser = Lark(grammar, start='sentence', parser='lalr')

class IntervalTransformer(Transformer):
    def __init__(self):
        self.anterior = None
        self.erro = False
        self.sentido = None
        self.interval_list = []

    def signal(self, items):
        self.sentido = items[0].value

    def intervals(self, items):
        return items

    def interval(self, items):
        start = float(items[1].children[0].value)
        end = float(items[3].children[0].value)

        self.interval_list.append(abs(end - start))

        if self.sentido == "+":
            CC1 = end > start
            CC2 = start >= self.anterior if self.anterior is not None else True
        else:
            CC1 = start > end
            CC2 = start <= self.anterior if self.anterior is not None else True

        if not CC1:
            print(f"CC1 Error: interval [{start}:{end}] is in the wrong order according to the sentence signal '{self.sentido}'")
            self.erro = True

        if not CC2:
            print(f"CC2 Error: interval [{start}:{end}] starts with {start} but previous interval ends with {self.anterior}")
            self.erro = True

        self.anterior = end

    def sentence_info(self):
        is_valid = not self.erro
        num_intervals = len(self.interval_list)
        largest_interval = max(self.interval_list) if self.interval_list else 0
        return is_valid, num_intervals, largest_interval


input_data = "- [1.0:9.0] [15.0:3.0]."
tree = parser.parse(input_data)
print(tree.pretty())

transformer = IntervalTransformer()
transformer.transform(tree)

is_valid, num_intervals, largest_interval = transformer.sentence_info()
print(f"\nIsValid: {is_valid} \nNumber of intervals: {num_intervals} \nLargest interval width: {largest_interval}")

sentence
  signal	-
  intervals
    interval
      [
      num	1.0
      :
      num	9.0
      ]
    interval
      [
      num	15.0
      :
      num	3.0
      ]
  .

CC1 Error: interval [1.0:9.0] is in the wrong order according to the sentence signal '-'
CC2 Error: interval [15.0:3.0] starts with 15.0 but previous interval ends with 9.0

IsValid: False 
Number of intervals: 2 
Largest interval width: 12.0
